In [4]:
import os
import tensorflow as tf
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import h5py
import zipfile

In [5]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2024-10-15 15:27:08--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.12.27, 172.217.15.251, 172.217.164.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.12.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   287MB/s    in 0.2s    

2024-10-15 15:27:09 (287 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [6]:
zip_object = zipfile.ZipFile(file="./cats_and_dogs_filtered.zip", mode="r")
zip_object.extractall("./")
zip_object.close()

In [7]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [8]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

In [9]:
IMG_SHAPE = (128,128,3)

In [10]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [14]:
#freezing the base model

base_model.trainable = False

In [19]:
#adding custom head to the model
base_model.output
#using gloabal avg pooling layer to handle the shape
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
prediction_layer = tf.keras.layers.Dense(units = 1 , activation='sigmoid')(global_avg_layer)

In [22]:
#defining the model
model = tf.keras.models.Model(inputs = base_model.input , outputs = prediction_layer)

In [24]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 32)     │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 64, 64, 32)     │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 64, 64, 32)     │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 64, 64, 32)     │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 64, 64, 32)     │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 64, 64, 16)     │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 64, 64, 16)     │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 64, 64, 96)     │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 64, 64, 96)     │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 64, 64, 96)     │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 65, 65, 96)     │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 32, 32, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [26]:
#compile the model
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss = 'binary_crossentropy',metrics = ['accuracy'])

In [27]:
#image data generator
train_gen = ImageDataGenerator(rescale = 1/255.)
validation_gen = ImageDataGenerator(rescale = 1/255.)




In [28]:
train_generator = train_gen.flow_from_directory(train_dir,target_size = (128,128),batch_size = 12, class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [29]:
validation_generator = validation_gen.flow_from_directory(validation_dir , target_size = (128,128),batch_size = 12  , class_mode= 'binary')

Found 1000 images belonging to 2 classes.


In [31]:
#transfer_learning

model.fit(train_generator , epochs = 5 ,validation_data = validation_generator)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


167/167 ━━━━━━━━━━━━━━━━━━━━ 43s 224ms/step - accuracy: 0.6437 - loss: 0.6421 - val_accuracy: 0.8610 - val_loss: 0.3679
Epoch 2/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 38s 208ms/step - accuracy: 0.8873 - loss: 0.3155 - val_accuracy: 0.9210 - val_loss: 0.2288
Epoch 3/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 34s 201ms/step - accuracy: 0.9279 - loss: 0.2127 - val_accuracy: 0.9370 - val_loss: 0.1752
Epoch 4/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 37s 220ms/step - accuracy: 0.9475 - loss: 0.1601 - val_accuracy: 0.9450 - val_loss: 0.1504
Epoch 5/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 33s 199ms/step - accuracy: 0.9484 - loss: 0.1362 - val_accuracy: 0.9490 - val_loss: 0.1357


In [35]:
test_loss , test_accuraccy = model.evaluate(validation_generator)

84/84 ━━━━━━━━━━━━━━━━━━━━ 13s 159ms/step - accuracy: 0.9389 - loss: 0.1367


In [36]:
test_accuraccy

0.9490000009536743

In [37]:
#fine tuning
#unfreeze few top layers

base_model.trainable = True

In [38]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 154


In [39]:
fine_tune_at = 100


In [40]:
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False


In [41]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),loss = 'binary_crossentropy',metrics = ['accuracy'])


In [42]:
model.fit(train_generator , epochs = 5 ,validation_data = validation_generator)


Epoch 1/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 72s 354ms/step - accuracy: 0.9130 - loss: 0.2199 - val_accuracy: 0.9380 - val_loss: 0.1523
Epoch 2/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 54s 322ms/step - accuracy: 0.9605 - loss: 0.0958 - val_accuracy: 0.9550 - val_loss: 0.1783
Epoch 3/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 62s 373ms/step - accuracy: 0.9816 - loss: 0.0538 - val_accuracy: 0.9310 - val_loss: 0.3061
Epoch 4/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 53s 317ms/step - accuracy: 0.9805 - loss: 0.0614 - val_accuracy: 0.9720 - val_loss: 0.1033
Epoch 5/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 53s 315ms/step - accuracy: 0.9915 - loss: 0.0274 - val_accuracy: 0.9710 - val_loss: 0.1504


In [43]:
test_loss , test_accuracy = model.evaluate(validation_generator)



84/84 ━━━━━━━━━━━━━━━━━━━━ 12s 140ms/step - accuracy: 0.9713 - loss: 0.1515


In [44]:
test_accuraccy

0.9490000009536743